In [ ]:
%pip install -U huggingface_hub

## Local Inference on GPU
Model page: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment, running this may bill your account above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, using the **auto** provider setting (automatically selects an available inference provider).

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="auto",
)

completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        },
        {
            "role": "assistant",
            "content": " The capital of France is Paris. It is one of the most famous cities in the world, known for its rich history, art, culture, and landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. Paris is also the political, economic, and cultural center of France."
        },
        {
            "role": "user",
            "content": "How confident are you?"
        },
        {
            "role": "assistant",
            "content": "  I am a model and my responses are based on the data I have been trained on. I strive to provide accurate and helpful information, but I don't have personal feelings or emotions. I don't have the ability to be confident or uncertain. I simply provide the information I have been programmed to know."
        },
    ],
)

print(completion.choices[0].message)
print('---')
print(completion.choices[0].message.content)

In [ ]:
# prompt: Can you do another chat completions call but this time — show log probabilities of top 10 tokens for each token inferred. And then the final message
import math, pandas as pd


completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
    logprobs=True,
    top_logprobs=5,
)
# print(completion.choices[0].message)
# completion.choices[0].logprobs

# logprobs = completion.choices[0].logprobs     # shortcut
# records = []

# for idx, logprob_item in enumerate(logprobs.content):
#   tok = logprob_item.token
#   lp = logprob_item.logprob
#   top = logprob_item.top_logprobs

#   chosen = f"{tok} ({math.exp(lp):.3f})"

#   # alts is now a list of Logprob objects, not a dictionary
#   alts_fmt = []

# for alt_logprob_item in top:
#   alts_fmt.append(f"{alt_logprob_item.token} ({math.exp(alt_logprob_item.logprob):.3f})")

# alts_fmt += [''] * (5 - len(alts_fmt))          # pad to 5

# records.append([idx, chosen, *alts_fmt])


for idx, (tok, lp, top) in enumerate(zip(
        logprobs.tokens,
        logprobs.token_logprobs,
        logprobs.top_logprobs)):

    chosen = f"{tok} ({math.exp(lp):.3f})"
    alts = sorted(top.items(), key=lambda kv: kv[1], reverse=True)[:5]
    alts_fmt = [f"{t} ({math.exp(lp_):.3f})" for t, lp_ in alts]
    alts_fmt += [''] * (5 - len(alts_fmt))          # pad to 5

    records.append([idx, chosen, *alts_fmt])

df = pd.DataFrame(records,
                  columns=["Idx", "Chosen (p)", "Alt-1", "Alt-2",
                           "Alt-3", "Alt-4", "Alt-5"])

print(df.to_string(index=False))



In [ ]:
# prompt: Write code that can redner the response from LLM, colour code each token/word based on probability. Higher to lower colours: [ #FFFFFF, #dbdbdb, #adadac, #696868, #333331, #000000]   Prompt, role=user "Write 3 paragraphs about Paris"

from IPython.display import display, HTML

def render_colored_text(completion):
    """
    Renders the text response from an LLM, coloring each token based on its probability.

    Args:
        completion: The completion object from the LLM client with logprobs enabled.
    """
    logprobs = completion.choices[0].logprobs
    if not logprobs or not logprobs.token_logprobs:
        print("Log probabilities not available in the completion.")
        print(completion.choices[0].message.content)
        return

    # Define color mapping for probabilities (higher to lower)
    # Mapping log_prob to a 0-1 range based on min/max log_probs
    colors = ["#000000", "#333331", "#696868", "#adadac", "#dbdbdb", "#FFFFFF"] # reversed for lower prob darker
    min_log_prob = min(logprobs.token_logprobs)
    max_log_prob = max(logprobs.token_logprobs)
    log_prob_range = max_log_prob - min_log_prob

    html_output = ""
    for token, log_prob in zip(logprobs.tokens, logprobs.token_logprobs):
        # Normalize log_prob to a 0-1 range for color mapping
        if log_prob_range > 0:
            normalized_prob = (log_prob - min_log_prob) / log_prob_range
        else:
            normalized_prob = 0.5 # Default to middle color if all log_probs are the same

        # Map normalized_prob to a color index
        color_index = int(normalized_prob * (len(colors) - 1))
        color = colors[color_index]

        # Encode HTML entities for special characters in tokens
        import html
        escaped_token = html.escape(token)

        html_output += f'<span style="color: {color};">{escaped_token}</span>'

    display(HTML(html_output))

# Render the response with colors
print("Rendering response with color coding based on token probability:")

completion_paris = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    messages=[
        {
            "role": "user",
            "content": "Write 3 paragraphs about Paris"
        }
    ],
    logprobs=True,
    top_logprobs=5,
)

render_colored_text(completion_paris)



## Temperature

In [ ]:
# prompt: Do same question "What is capital of france" with chat completion — allow controlling other factors like output tokens, temperature

completion = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.3",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
    max_tokens=1,  # Set the maximum number of output tokens
    temperature=0 # Set the temperature for creativity (0.0 to 1.0)
)

completion.choices[0].message.content